In [45]:
import networkx as nx
import scripts.read_graph as rg
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [4]:
def read(path):

    MG = nx.read_gexf(path)

    for node in MG.nodes(data=True):
        for key, value in MG.edge[node[0]].items():
            for k, v in value.items():
                if type(v['attr']) == str:
                    MG.edge[node[0]][key][k]['attr'] = ast.literal_eval(v['attr'])

    for node in MG.nodes(data=True):
        if isinstance(MG.node[node[0]]['location'], str):
            MG.node[node[0]]['location'] = ast.literal_eval(MG.node[node[0]]['location'])
            if not isinstance(MG.node[node[0]]['location'], dict):
                print('NAO FUNCIONA')
                break
        if len(node[1]) == 3:
            if type(MG.node[node[0]]['viagens']) == str:
                MG.node[node[0]]['viagens'] = ast.literal_eval(MG.node[node[0]]['viagens'])

    return MG

In [9]:
MG = rg.read('graphs/sp_final.gexf')

Não deixar o fera ficar entrando e saindo de busao

In [13]:
def getpath(nested_dict, value, prepath=()):
    for k, v in nested_dict.items():
        path = prepath + (k,)
        if v == value: # found value
            return path
        elif hasattr(v, 'items'): # v is a dict
            p = getpath(v, value, path) # recursive call
            if p is not None:
                return p

In [75]:
class Walker():
    
    def __init__(self, graph, stop_id, travel_time, current_time):
        self.MG = graph
        self.stop_id = stop_id
        self.total_time_travel = travel_time * 60 # minutos para segundos
        self.traveled_time = 0
        self.visited_set = []
        self.tentative_value = {}
        self.current_time = current_time * 60 * 60 # hora para segundo
        
        self.calculate_location = lambda x: (MG.node[x]['location']['lat'], MG.node[x]['location']['long'])# tuple
        self.current_location = self.calculate_location(stop_id)
        
    def isochrone(self):
        i=0
        while 1:
            print('\n{}: '.format(i))
            self.find_neighbours()

            minimum, min_key = -1, 0
            for key, values in self.tentative_value.items():
                if key not in self.visited_set:
                    if minimum == -1:
                        minimum = values['travel_time']
                        min_key = key
                    elif values['travel_time'] < minimum:
                        minimum = values['travel_time']
                        min_key = key
            
            
            self.stop_id = min_key
            self.traveled_time = self.traveled_time + minimum
            self.current_time = self.current_time + minimum
            
            print('\n--------\nBusao:', self.tentative_value[min_key]['linha'])
            #print('Tentative_Values', self.tentative_value)
            #print('Visited set', self.visited_set)
            print('Traveled Time: ',self.traveled_time)
            if self.traveled_time > self.total_time_travel:
                print('FINAL:', self.stop_id)
                break
            i=i+1
            
    def find_neighbours(self):
        """Given a stop, find neighbours and calculate travel time. 
        Two types of neighbours: connectionable and walkable
        """
        #pprint('StopId: ', self.stop_id)
        #pprint(self.current_time)
        # get connection neighbours
        linha_atual = 0
        if len(self.visited_set) > 0:
            linha_atual = self.tentative_value[self.stop_id]['linha']
            # print('Linha Atual:', linha_atual)
            
        for conn_neigh_key, conn_neigh_value in self.MG.node[self.stop_id]['viagens'].items():
            
            #print('TripId', conn_neigh_key)
            # get next travel time
            # print("------\nLine Name: ", conn_neigh_key, "\n")
            #print(conn_neigh_value)
            travel_time =  (min(conn_neigh_value,
                              key=lambda x: abs(x-self.current_time) if x-self.current_time<=0 else False) -
                            self.current_time)
            """
            if conn_neigh_key == linha_atual:
                pass
                print("----Current Line:", travel_time)
            else:
                pass
                print("Other Line:", travel_time)
            """
            # have to be greater than current_time
            if travel_time < 0:
                continue
            
            # get time to next stop  
            output = getpath(self.MG.edge[self.stop_id], conn_neigh_key)
            if output == None:
                continue
            stop_id_next, trip_key, *rest = output
            
            if stop_id_next in self.visited_set:
                continue
            #print('StopIdNext: ',stop_id_next)
            trip_time = self.MG.edge[self.stop_id][stop_id_next][trip_key]['attr']['tempo']
            travel_time = travel_time + trip_time
            
            # print("Trip Time: ", trip_time)
            # print("Total Time: ", travel_time)
            # print('Stop Next', stop_id_next, "\n")
            
            # check travel time to next stop and replace if smaller
            if stop_id_next in self.tentative_value:
                if travel_time < self.tentative_value[stop_id_next]['travel_time']:
                    self.tentative_value[stop_id_next] = {'travel_time': travel_time,
                                                          'trip_key': trip_key,
                                                          'previous_stop': self.stop_id,
                                                        'linha': conn_neigh_key}
            else:
                self.tentative_value[stop_id_next] = {'travel_time': travel_time,
                                                      'trip_key': trip_key,
                                                      'previous_stop': self.stop_id,
                                                     'linha': conn_neigh_key}
   
            #print(self.tentative_value)
    
        self.visited_set.append(self.stop_id)
        
        # get walkable neighbours
        

In [76]:
node_id = '706304'
walker = Walker(MG, node_id, travel_time=60, current_time=21)
walker.isochrone()
print(walker.stop_id)


0: 

--------
Busao: 874T-10-0
Traveled Time:  124

1: 

--------
Busao: 874T-10-0
Traveled Time:  231

2: 

--------
Busao: 874T-10-0
Traveled Time:  338

3: 

--------
Busao: 874T-10-0
Traveled Time:  445

4: 

--------
Busao: 874T-10-0
Traveled Time:  552

5: 

--------
Busao: 874T-10-0
Traveled Time:  659

6: 

--------
Busao: 874T-10-0
Traveled Time:  766

7: 

--------
Busao: 874T-10-0
Traveled Time:  873

8: 

--------
Busao: 874T-10-0
Traveled Time:  980

9: 

--------
Busao: 874T-10-0
Traveled Time:  1087

10: 

--------
Busao: 874T-10-0
Traveled Time:  1194

11: 

--------
Busao: 874T-10-0
Traveled Time:  1301

12: 

--------
Busao: 874T-10-0
Traveled Time:  1408

13: 

--------
Busao: 874T-10-0
Traveled Time:  1515

14: 

--------
Busao: 874T-10-0
Traveled Time:  1622

15: 

--------
Busao: 874T-10-0
Traveled Time:  1729

16: 

--------
Busao: 874T-10-0
Traveled Time:  1836

17: 

--------
Busao: 874T-10-0
Traveled Time:  1943

18: 

--------
Busao: 874T-10-0
Traveled Time:

In [86]:
stops = [MG.node[i] for i in walker.visited_set]
stops[0]['location']

{'lat': -23.565893, 'long': -46.650874}

In [97]:
import folium
COORD = (stops[0]['location']['lat'],stops[0]['location']['long']) # sp coordinates
mapa = folium.Map(location=COORD, zoom_start=11)

for i, stop in enumerate(stops[1:]):
    if i == 0:
        color = '#ff0000'
        radius=5
    else:
        color = '#45647d'
        radius=1
    
    folium.RegularPolygonMarker(
        location = [stop['location']['lat'],stop['location']['long']],
        fill_color=color,
        number_of_sides=100,
        radius=radius,
        ).add_to(mapa)
mapa